In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

In [87]:
h = [1, 0, 0 ,0] # dim (4)
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

Single RNN

In [88]:
# output dim (2)
cell = nn.RNN(input_size=4, hidden_size=2, batch_first=True)

In [95]:
inputs = torch.tensor([[h]], dtype=torch.float) # rank = (1, 1, 4)

In [96]:
hidden = torch.randn(1, 1, 2)

In [97]:
out, hidden = cell(inputs, hidden)

In [98]:
out

tensor([[[0.0886, 0.5660]]], grad_fn=<TransposeBackward0>)

# Unfolding to `n` sequences

In [103]:
inputs = [[h, e, l, l, o]]
inputs = torch.tensor(inputs, dtype=torch.float)

In [104]:
inputs.size()

torch.Size([1, 5, 4])

In [105]:
hidden = torch.randn(1, 1, 2)

In [102]:
cell = nn.RNN(input_size=4, hidden_size=2, batch_first=True)

In [106]:
out, hidden = cell(inputs, hidden)
out

tensor([[[ 0.8379,  0.1169],
         [ 0.5100,  0.3034],
         [ 0.8857,  0.0452],
         [ 0.8154, -0.0911],
         [ 0.8023,  0.1019]]], grad_fn=<TransposeBackward0>)

In [107]:
out.size()

torch.Size([1, 5, 2])

# Batching input

<img src="RNN.png" />

In [112]:
hidden_size = 2
sequence_length = 5
batch_size = 3

In [110]:
inputs = [[h, e, l, l, o], 
          [e, o, l, l, l],
          [l, l, e, e, l]]
inputs = torch.tensor(inputs, dtype=torch.float)

In [111]:
inputs

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]])

In [115]:
inputs.size() # (batch, seq, one_hot_size)

torch.Size([3, 5, 4])

`hidden_size` changed to `3` (batch_size)

In [116]:
hidden = torch.randn(1, 3, 2)

In [ ]:
out, 

In [73]:
x_data = [0, 1, 0, 2, 3, 3] # hihell
one_hot_dict = {
    'h': [1, 0, 0, 0, 0],
    'i': [0, 1, 0, 0, 0],
    'e': [0, 0, 1, 0, 0],
    'l': [0, 0, 0, 1, 0],
    'o': [0, 0, 0, 0, 1],
}
one_hot_lookup = [
    [1, 0, 0, 0, 0], # 0 h
    [0, 1, 0, 0, 0], # 1 i
    [0, 0, 1, 0, 0], # 2 e
    [0, 0, 0, 1, 0], # 3 l
    [0, 0, 0, 0, 1], # 4 o
]
y_data = [1, 0, 2, 3, 3, 4] # ihello
x_one_hot = [one_hot_lookup[x] for x in x_data]

In [74]:
num_classes = 5
input_size = 5  # one_hot size
hidden_size = 5 # output from the LSTM. 5 to directly predict one-hot
batch_size = 1  # one sentence
sequence_length = 1 # Let's do one by one
num_layers = 1  # one-layer rnn

In [75]:
inputs = 'hihell'
labels = 'ihello'
ans = [one_hot_dict[i] for i in inputs]
inputs = torch.tensor(ans, dtype=torch.float)
tmp = [one_hot_dict[i] for i in labels]
labels = torch.tensor(tmp, dtype=torch.long)

# 1. Model

In [76]:
class Model(nn.Module):
    def __init__(self, 
                 input_size=5, 
                 hidden_size=5, 
                 num_layers=1, 
                 batch_size=1, 
                 sequence_length=1):
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size, 
                          hidden_size=hidden_size, batch_first=True)
    
    def forward(self, x, hidden):
        # Reshape input in (batch_size, sequence_length, input_size)
        x = x.view(batch_size, sequence_length, input_size)
        hidden = hidden.view(batch_size, sequence_length, input_size)

#         import ipdb; ipdb.set_trace()        
        out, hidden = self.rnn(x, hidden)
        out = out.view(-1, num_classes)
        return hidden, out
    
    def init_hidden(self):
        return torch.zeros(num_layers, batch_size, hidden_size)

# 2. Criterion & Loss

In [77]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# 3. Training

In [80]:
for epoch in range(100):
    outputs = model(inputs, hidden)
    optimizer.zero_grad()

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    _, idx = outputs.max(1)
    print(idx)

RuntimeError: shape '[1, 1, 5]' is invalid for input of size 30